In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.compose import make_column_transformer,ColumnTransformer
from sklearn.pipeline import make_pipeline,Pipeline

In [2]:
train = pd.read_csv('Train_Data.csv')
test = pd.read_csv('Test_Data.csv')

In [3]:
train.head(30)

,date,campaign,adgroup,ad,impressions,clicks,cost,conversions,revenue
0,01-08-2020,campaign 1,adgroup 1,ad 1,24,6,0.08,0,0.00
1,01-08-2020,campaign 1,adgroup 2,ad 1,1,0,0.00,0,0.00
2,01-08-2020,campaign 1,adgroup 3,ad 1,13,4,0.04,0,0.00
3,01-08-2020,campaign 1,adgroup 4,ad 1,5,4,0.08,0,0.00
4,01-08-2020,campaign 1,adgroup 1,ad 2,247,126,1.29,4,925.71
5,01-08-2020,campaign 1,adgroup 2,ad 2,7,2,0.04,0,0.00
6,01-08-2020,campaign 1,adgroup 4,ad 2,6,3,0.05,0,0.00
7,01-08-2020,campaign 1,adgroup 1,ad 3,129,52,0.58,0,0.00
8,01-08-2020,campaign 1,adgroup 3,ad 3,104,56,0.56,0,0.00
9,01-08-2020,campaign 1,adgroup 4,ad 3,6,5,0.05,0,0.00


In [4]:
np.random.seed(42)
train = train.reindex(np.random.permutation(train.index))
train.reset_index(inplace=True, drop=True)

In [5]:
train.head()

,date,campaign,adgroup,ad,impressions,clicks,cost,conversions,revenue
0,29-10-2020,campaign 1,adgroup 1,ad 3,58,26,6.86,0,0.00
1,02-08-2020,campaign 1,adgroup 1,ad 1,21,8,0.09,0,0.00
2,14-08-2020,campaign 1,adgroup 1,ad 1,61,32,0.35,2,197.59
3,14-12-2020,campaign 1,adgroup 4,ad 3,4,2,0.02,0,0.00
4,03-02-2021,campaign 1,adgroup 3,ad 8,98,47,2.67,0,0.00


In [6]:
train.campaign.value_counts()

campaign 1    4571
Name: campaign, dtype: int64

In [7]:
test.campaign.value_counts()

campaign 1    318
Name: campaign, dtype: int64

In [8]:
train.ad.value_counts()

ad 3     617
ad 1     587
ad 2     512
ad 7     461
ad 32    377
        ... 
ad 47      4
ad 69      4
ad 36      4
ad 31      4
ad 51      3
Name: ad, Length: 70, dtype: int64

In [9]:
test.ad.value_counts()

ad 3     44
ad 1     42
ad 7     33
ad 2     30
ad 56    24
ad 72    22
ad 55    19
ad 10    13
ad 73    13
ad 74    12
ad 8     11
ad 6     11
ad 71    11
ad 4     11
ad 5     11
ad 75    11
Name: ad, dtype: int64

In [10]:
train.ad.unique()

array(['ad 3', 'ad 1', 'ad 8', 'ad 6', 'ad 26', 'ad 38', 'ad 56', 'ad 55',
       'ad 67', 'ad 54', 'ad 5', 'ad 7', 'ad 21', 'ad 2', 'ad 10',
       'ad 61', 'ad 42', 'ad 17', 'ad 63', 'ad 4', 'ad 32', 'ad 46',
       'ad 27', 'ad 28', 'ad 12', 'ad 70', 'ad 60', 'ad 48', 'ad 19',
       'ad 25', 'ad 52', 'ad 53', 'ad 11', 'ad 22', 'ad 18', 'ad 40',
       'ad 64', 'ad 16', 'ad 15', 'ad 34', 'ad 13', 'ad 66', 'ad 44',
       'ad 59', 'ad 47', 'ad 51', 'ad 49', 'ad 58', 'ad 69', 'ad 43',
       'ad 45', 'ad 50', 'ad 9', 'ad 68', 'ad 23', 'ad 30', 'ad 57',
       'ad 35', 'ad 65', 'ad 24', 'ad 14', 'ad 29', 'ad 33', 'ad 41',
       'ad 62', 'ad 31', 'ad 37', 'ad 36', 'ad 20', 'ad 39'], dtype=object)

In [11]:
train.adgroup.value_counts()

adgroup 3    1672
adgroup 1    1314
adgroup 4     858
adgroup 2     727
Name: adgroup, dtype: int64

In [12]:
test.adgroup.value_counts()

adgroup 3    111
adgroup 1     90
adgroup 2     63
adgroup 4     54
Name: adgroup, dtype: int64

In [13]:
train.describe()

,impressions,clicks,cost,conversions,revenue
count,4571.000000,4571.000000,4571.000000,4571.000000,4571.000000
mean,92.635747,43.340844,2.800834,1.339094,231.865340
std,198.349737,100.164913,14.361618,5.223922,1091.742763
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,6.000000,2.000000,0.040000,0.000000,0.000000
50%,16.000000,7.000000,0.210000,0.000000,0.000000
75%,84.000000,38.000000,1.350000,1.000000,92.600000
max,3239.000000,1771.000000,531.250000,94.000000,20515.410000


In [14]:
test.describe()

,cost,impressions,clicks,conversions
count,318.000000,318.000000,318.000000,318.000000
mean,0.381352,67.144654,29.720126,0.805031
std,0.591480,101.078870,45.200014,1.802480
min,0.000000,1.000000,0.000000,0.000000
25%,0.020000,4.000000,2.000000,0.000000
50%,0.105000,20.000000,9.000000,0.000000
75%,0.490000,103.000000,40.750000,1.000000
max,3.530000,589.000000,276.000000,15.000000


In [15]:
train.shape

(4571, 9)

In [16]:
X_train_orig = train.drop(['revenue','campaign'],axis=1)
X_test_orig = test.copy(deep=True)
X_test_orig = X_test_orig.drop('campaign',axis=1)
Y_train_orig = train['revenue']

In [17]:
X_train_orig.head()

,date,adgroup,ad,impressions,clicks,cost,conversions
0,29-10-2020,adgroup 1,ad 3,58,26,6.86,0
1,02-08-2020,adgroup 1,ad 1,21,8,0.09,0
2,14-08-2020,adgroup 1,ad 1,61,32,0.35,2
3,14-12-2020,adgroup 4,ad 3,4,2,0.02,0
4,03-02-2021,adgroup 3,ad 8,98,47,2.67,0


In [18]:
train.isna().sum()

date           0
campaign       0
adgroup        0
ad             0
impressions    0
clicks         0
cost           0
conversions    0
revenue        0
dtype: int64

In [19]:
test.isna().sum()

date           0
campaign       0
adgroup        0
ad             0
cost           0
impressions    0
clicks         0
conversions    0
dtype: int64

In [20]:
X_train_orig['date'].value_counts()

30-10-2020    35
29-10-2020    34
22-12-2020    34
24-12-2020    33
31-10-2020    32
              ..
13-09-2020    10
11-02-2021    10
28-08-2020     8
29-09-2020     8
31-08-2020     8
Name: date, Length: 212, dtype: int64

In [21]:
X_train_orig['date'] = pd.to_datetime(X_train_orig.date)
X_test_orig['date'] = pd.to_datetime(X_test_orig.date)

In [22]:
X_train_orig.dtypes

date           datetime64[ns]
adgroup                object
ad                     object
impressions             int64
clicks                  int64
cost                  float64
conversions             int64
dtype: object

In [23]:
X_train_orig.head()

,date,adgroup,ad,impressions,clicks,cost,conversions
0,2020-10-29,adgroup 1,ad 3,58,26,6.86,0
1,2020-02-08,adgroup 1,ad 1,21,8,0.09,0
2,2020-08-14,adgroup 1,ad 1,61,32,0.35,2
3,2020-12-14,adgroup 4,ad 3,4,2,0.02,0
4,2021-03-02,adgroup 3,ad 8,98,47,2.67,0


In [24]:
X_train_orig['month'] = X_train_orig['date'].dt.month
X_test_orig['month'] = X_test_orig['date'].dt.month
X_train_orig['day'] = X_train_orig['date'].dt.day
X_test_orig['day'] = X_test_orig['date'].dt.day

In [25]:
X_train_orig.head()

,date,adgroup,ad,impressions,clicks,cost,conversions,month,day
0,2020-10-29,adgroup 1,ad 3,58,26,6.86,0,10,29
1,2020-02-08,adgroup 1,ad 1,21,8,0.09,0,2,8
2,2020-08-14,adgroup 1,ad 1,61,32,0.35,2,8,14
3,2020-12-14,adgroup 4,ad 3,4,2,0.02,0,12,14
4,2021-03-02,adgroup 3,ad 8,98,47,2.67,0,3,2


In [26]:
X_test_orig.head()

,date,adgroup,ad,cost,impressions,clicks,conversions,month,day
0,2021-01-03,adgroup 1,ad 1,0.58,121,49,1,1,3
1,2021-01-03,adgroup 3,ad 1,0.17,22,12,0,1,3
2,2021-01-03,adgroup 4,ad 1,0.05,5,3,0,1,3
3,2021-01-03,adgroup 2,ad 1,0.01,2,1,0,1,3
4,2021-01-03,adgroup 2,ad 2,0.01,3,1,0,1,3


In [27]:
X_train = X_train_orig.drop('date',axis=1)
X_test = X_test_orig.drop('date',axis=1)

In [28]:
col_trans = make_column_transformer((OneHotEncoder(), ['adgroup','ad']) , remainder='passthrough')

In [29]:
X_train.dtypes

adgroup         object
ad              object
impressions      int64
clicks           int64
cost           float64
conversions      int64
month            int64
day              int64
dtype: object

In [30]:
X_train.head()

,adgroup,ad,impressions,clicks,cost,conversions,month,day
0,adgroup 1,ad 3,58,26,6.86,0,10,29
1,adgroup 1,ad 1,21,8,0.09,0,2,8
2,adgroup 1,ad 1,61,32,0.35,2,8,14
3,adgroup 4,ad 3,4,2,0.02,0,12,14
4,adgroup 3,ad 8,98,47,2.67,0,3,2


In [31]:
X_test.head()

,adgroup,ad,cost,impressions,clicks,conversions,month,day
0,adgroup 1,ad 1,0.58,121,49,1,1,3
1,adgroup 3,ad 1,0.17,22,12,0,1,3
2,adgroup 4,ad 1,0.05,5,3,0,1,3
3,adgroup 2,ad 1,0.01,2,1,0,1,3
4,adgroup 2,ad 2,0.01,3,1,0,1,3


In [32]:
X_test = X_test[['adgroup','ad','impressions','clicks','cost','conversions','month','day']]

In [33]:
obj_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']
num_cols = [col for col in X_train.columns if X_train[col].dtype != 'object']

In [34]:
print(num_cols)

['impressions', 'clicks', 'cost', 'conversions', 'month', 'day']


In [35]:
num_trans = StandardScaler()
cat_trans = OneHotEncoder(handle_unknown='ignore')
pre_pros = ColumnTransformer(transformers = [ ('num',num_trans,num_cols) , ('cat',cat_trans,obj_cols) ])

In [36]:
linreg_pipe= Pipeline( steps=[ ('preprocessor',pre_pros) , ('classifier',LinearRegression()) ] )

In [37]:
linreg_pipe.fit(X_train,Y_train_orig)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['impressions', 'clicks',
                                                   'cost', 'conversions',
                                                   'month', 'day']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['adgroup', 'ad'])])),
                ('classifier', LinearRegression())])

In [38]:
pred_test = linreg_pipe.predict(X_test)

In [39]:
pred_train = linreg_pipe.predict(X_train)
print(mean_squared_error(Y_train_orig,pred_train))

59756.26653337064


In [40]:
cross_val_score(linreg_pipe,X_train,Y_train_orig).mean()

0.9332317606924949

In [41]:
predictions = pd.DataFrame(columns=["revenue"],data=pred_test)
predictions.to_csv("prediction_results_linreg.csv", index = False)

In [42]:
X_train.head()

,adgroup,ad,impressions,clicks,cost,conversions,month,day
0,adgroup 1,ad 3,58,26,6.86,0,10,29
1,adgroup 1,ad 1,21,8,0.09,0,2,8
2,adgroup 1,ad 1,61,32,0.35,2,8,14
3,adgroup 4,ad 3,4,2,0.02,0,12,14
4,adgroup 3,ad 8,98,47,2.67,0,3,2


In [43]:
from xgboost import XGBRegressor


In [44]:
XGB_pipe= Pipeline( steps=[ ('preprocessor',pre_pros) , ('classifier',XGBRegressor(max_depth=4,learning_rate=0.004,n_estimators=500)) ] )

In [45]:
XGB_pipe.fit(X_train,Y_train_orig)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['impressions', 'clicks',
                                                   'cost', 'conversions',
                                                   'month', 'day']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['adgroup', 'ad'])])),
                ('classifier',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, gamma=0, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='', learning_rate=0.004,
                              max_delta_step=0, max_depth=4, min_child_weight=1,
   

In [46]:
cross_val_score(XGB_pipe,X_train,Y_train_orig).mean()

0.9097480366222446

In [47]:
pred_test_XGB = XGB_pipe.predict(X_test)
predictions_XGB = pd.DataFrame(columns=["revenue"],data=pred_test_XGB)
predictions_XGB.to_csv("prediction_results_XGB.csv", index = False)

In [48]:
train_set_comparison = train.copy(deep=True)
pred_train_XGB = XGB_pipe.predict(X_train)
predictions_XGB_train = pd.DataFrame(columns=["revenue"],data=pred_train_XGB)
train_set_comparison['prediction'] = predictions_XGB_train['revenue']
train_set_comparison.head()

,date,campaign,adgroup,ad,impressions,clicks,cost,conversions,revenue,prediction
0,29-10-2020,campaign 1,adgroup 1,ad 3,58,26,6.86,0,0.00,0.304052
1,02-08-2020,campaign 1,adgroup 1,ad 1,21,8,0.09,0,0.00,0.304052
2,14-08-2020,campaign 1,adgroup 1,ad 1,61,32,0.35,2,197.59,285.312317
3,14-12-2020,campaign 1,adgroup 4,ad 3,4,2,0.02,0,0.00,0.304052
4,03-02-2021,campaign 1,adgroup 3,ad 8,98,47,2.67,0,0.00,0.304052


In [49]:
train_set_comparison['Error'] = train_set_comparison['revenue'] - train_set_comparison['prediction']
train_set_comparison.head()

,date,campaign,adgroup,ad,impressions,clicks,cost,conversions,revenue,prediction,Error
0,29-10-2020,campaign 1,adgroup 1,ad 3,58,26,6.86,0,0.00,0.304052,-0.304052
1,02-08-2020,campaign 1,adgroup 1,ad 1,21,8,0.09,0,0.00,0.304052,-0.304052
2,14-08-2020,campaign 1,adgroup 1,ad 1,61,32,0.35,2,197.59,285.312317,-87.722317
3,14-12-2020,campaign 1,adgroup 4,ad 3,4,2,0.02,0,0.00,0.304052,-0.304052
4,03-02-2021,campaign 1,adgroup 3,ad 8,98,47,2.67,0,0.00,0.304052,-0.304052


In [50]:
train_set_comparison[train_set_comparison.Error > 2500]

,date,campaign,adgroup,ad,impressions,clicks,cost,conversions,revenue,prediction,Error
911,10-10-2020,campaign 1,adgroup 3,ad 31,1363,809,36.80,85,17863.91,15014.951172,2848.958828
1686,26-11-2020,campaign 1,adgroup 1,ad 44,1895,1031,186.26,56,12089.80,9104.467773,2985.332227
1709,08-09-2020,campaign 1,adgroup 3,ad 18,1356,714,24.51,36,12460.44,9772.419922,2688.020078
1822,08-10-2020,campaign 1,adgroup 1,ad 28,1507,761,41.92,71,18476.84,15014.951172,3461.888828
2497,09-09-2020,campaign 1,adgroup 1,ad 19,2101,1147,24.95,61,18761.60,15075.775391,3685.824609
2781,26-11-2020,campaign 1,adgroup 1,ad 43,2151,1068,121.52,82,16324.10,12677.459961,3646.640039
2850,10-09-2020,campaign 1,adgroup 1,ad 20,1657,944,18.33,91,19149.05,15075.775391,4073.274609
3204,10-09-2020,campaign 1,adgroup 3,ad 20,2009,1024,35.16,82,20515.41,15075.775391,5439.634609
3371,27-11-2020,campaign 1,adgroup 1,ad 43,3239,1771,531.25,94,15299.90,12523.489258,2776.410742
3965,09-09-2020,campaign 1,adgroup 3,ad 19,1361,713,22.75,47,14862.93,11621.271484,3241.658516


In [51]:
X_train.head()

,adgroup,ad,impressions,clicks,cost,conversions,month,day
0,adgroup 1,ad 3,58,26,6.86,0,10,29
1,adgroup 1,ad 1,21,8,0.09,0,2,8
2,adgroup 1,ad 1,61,32,0.35,2,8,14
3,adgroup 4,ad 3,4,2,0.02,0,12,14
4,adgroup 3,ad 8,98,47,2.67,0,3,2


In [52]:
X_train_nodate = X_train.drop(['month','day'],axis=1)
X_train_nodate.head()

,adgroup,ad,impressions,clicks,cost,conversions
0,adgroup 1,ad 3,58,26,6.86,0
1,adgroup 1,ad 1,21,8,0.09,0
2,adgroup 1,ad 1,61,32,0.35,2
3,adgroup 4,ad 3,4,2,0.02,0
4,adgroup 3,ad 8,98,47,2.67,0


In [53]:
num_cols_new=np.array(['impressions', 'clicks', 'cost', 'conversions'])

In [54]:
num_trans = StandardScaler()
cat_trans = OneHotEncoder(handle_unknown='ignore')
pre_pros = ColumnTransformer(transformers = [ ('num',num_trans,num_cols_new) , ('cat',cat_trans,obj_cols) ])
XGB_pipe= Pipeline( steps=[ ('preprocessor',pre_pros) , ('classifier',XGBRegressor(max_depth=4,learning_rate=0.004,n_estimators=500)) ] )
XGB_pipe.fit(X_train_nodate,Y_train_orig)
cross_val_score(XGB_pipe,X_train_nodate,Y_train_orig).mean()

0.8960811209992505

In [55]:
num_cols=[col for col in X_train.columns if X_train[col].dtype != 'object']
num_cols

['impressions', 'clicks', 'cost', 'conversions', 'month', 'day']

In [56]:
obj_cols

['adgroup', 'ad']

In [57]:
obj_cols_new=obj_cols.copy()

In [58]:
obj_cols_new.remove('ad')
obj_cols_new.remove('adgroup')
obj_cols

['adgroup', 'ad']

In [59]:
obj_cols_new

[]

In [60]:
X_train_noad = X_train.drop(['ad'],axis=1)
num_trans = StandardScaler()
cat_trans = OneHotEncoder(handle_unknown='ignore')
pre_pros = ColumnTransformer(transformers = [ ('num',num_trans,num_cols) , ('cat',cat_trans,obj_cols_new) ])
XGB_pipe= Pipeline( steps=[ ('preprocessor',pre_pros) , ('classifier',XGBRegressor(max_depth=4,learning_rate=0.004,n_estimators=600,min_child_weight=4)) ] )
XGB_pipe.fit(X_train_noad,Y_train_orig)
cross_val_score(XGB_pipe,X_train,Y_train_orig).mean()

0.9295702674968066

In [61]:
X_test.drop(['ad'],axis=1,inplace=True)

In [62]:
pred_test_XGB = XGB_pipe.predict(X_test)
predictions_XGB = pd.DataFrame(columns=["revenue"],data=pred_test_XGB)
predictions_XGB.to_csv("newrev7.csv", index = False)

In [63]:
X_train_new,X_test_new,Y_train_new,Y_test_new = train_test_split(X_train_noad,Y_train_orig,test_size=0.2)


In [64]:
XGB_pipe.fit(X_train_new,Y_train_new)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['impressions', 'clicks',
                                                   'cost', 'conversions',
                                                   'month', 'day']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  [])])),
                ('classifier',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, gamma=0, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='', learning_rate=0.004,
                              max_delta_step=0, max_depth=4, min_child_weight=4,
                  

In [65]:
pred = XGB_pipe.predict(X_train_new)

In [66]:
print(np.sqrt(mean_squared_error(Y_train_new,pred)))

210.41572086749454


In [67]:
predn = XGB_pipe.predict(X_test_new)

In [68]:
print(np.sqrt(mean_squared_error(Y_test_new,predn)))

278.73372141301644


In [69]:
X_train_noad.shape

(4571, 7)

In [70]:
from tensorflow.keras import regularizers


In [71]:
X_test = pd.DataFrame(data=X_test)

In [72]:
X_train_noad = pd.DataFrame(data=X_train_noad)
X_train.head()

,adgroup,ad,impressions,clicks,cost,conversions,month,day
0,adgroup 1,ad 3,58,26,6.86,0,10,29
1,adgroup 1,ad 1,21,8,0.09,0,2,8
2,adgroup 1,ad 1,61,32,0.35,2,8,14
3,adgroup 4,ad 3,4,2,0.02,0,12,14
4,adgroup 3,ad 8,98,47,2.67,0,3,2


In [73]:
X_train_noad = pd.get_dummies(X_train_noad, columns=["adgroup"])
X_train_noad.head()

,impressions,clicks,cost,conversions,month,day,adgroup_adgroup 1,adgroup_adgroup 2,adgroup_adgroup 3,adgroup_adgroup 4
0,58,26,6.86,0,10,29,1,0,0,0
1,21,8,0.09,0,2,8,1,0,0,0
2,61,32,0.35,2,8,14,1,0,0,0
3,4,2,0.02,0,12,14,0,0,0,1
4,98,47,2.67,0,3,2,0,0,1,0


In [74]:
X_test = pd.get_dummies(X_test, columns=["adgroup"])
X_test.head()

,impressions,clicks,cost,conversions,month,day,adgroup_adgroup 1,adgroup_adgroup 2,adgroup_adgroup 3,adgroup_adgroup 4
0,121,49,0.58,1,1,3,1,0,0,0
1,22,12,0.17,0,1,3,0,0,1,0
2,5,3,0.05,0,1,3,0,0,0,1
3,2,1,0.01,0,1,3,0,1,0,0
4,3,1,0.01,0,1,3,0,1,0,0


In [76]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from tensorflow.keras import regularizers

from sklearn.metrics import mean_squared_error
from math import sqrt

model = Sequential()
model.add(Dense(500, input_dim=10, activation= "relu",activity_regularizer=regularizers.l1(1e-4)))
model.add(Dense(100, activation= "relu",activity_regularizer=regularizers.l1(1e-4)))
model.add(Dense(50, activation= "relu",activity_regularizer=regularizers.l1(1e-4)))
model.add(Dense(1))

In [77]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss= "mean_absolute_error" , metrics=["mean_absolute_error"])
model.fit(X_train_noad,Y_train_orig, epochs=600,validation_split=0.15)

Epoch 1/600
122/122 [==============================] - 1s 5ms/step - loss: 216.0095 - mean_absolute_error: 215.7970 - val_loss: 210.6014 - val_mean_absolute_error: 210.3605
Epoch 2/600
122/122 [==============================] - 0s 3ms/step - loss: 182.5198 - mean_absolute_error: 182.2128 - val_loss: 188.4561 - val_mean_absolute_error: 188.1243
Epoch 3/600
122/122 [==============================] - 0s 3ms/step - loss: 171.8113 - mean_absolute_error: 171.4398 - val_loss: 184.6147 - val_mean_absolute_error: 184.2557
Epoch 4/600
122/122 [==============================] - 0s 4ms/step - loss: 169.1396 - mean_absolute_error: 168.7548 - val_loss: 181.0842 - val_mean_absolute_error: 180.7163
Epoch 5/600
122/122 [==============================] - 0s 3ms/step - loss: 167.0538 - mean_absolute_error: 166.6544 - val_loss: 178.2736 - val_mean_absolute_error: 177.8920
Epoch 6/600
122/122 [==============================] - 0s 3ms/step - loss: 165.3252 - mean_absolute_error: 164.9118 - val_loss: 175.559

In [78]:
prednn = model.predict(X_test)
predictions_nn = pd.DataFrame(columns=["revenue"],data=prednn)
predictions_nn.to_csv("newrev8.csv", index = False)